In [1]:
import numpy as np

# Inverse kinematic

## Symbol calculations

In [2]:
import sympy
sympy.init_printing()

In [3]:
Rx, Ry = sympy.symbols("R_x, R_y")
rx, ry = sympy.symbols("r_x, r_y")
vx, vy, w = sympy.symbols("v_x, v_y, \omega")

In [4]:
V = sympy.Matrix([vx, vy, 0]) + sympy.Matrix([0, 0, w]).cross(sympy.Matrix([Rx, Ry, 0]))
V_m = sympy.Matrix(V[0:2]).dot(sympy.Matrix([rx, ry]))
V_m = sympy.expand(V_m)

In [5]:
V_m

## Numerical calculations

In [8]:
Rx1 = 175.27
Ry1 = 120.27 / 2
a = np.pi / 4
rwheel = 30

rx1 = np.cos(a)
ry1 = np.sin(a)
angles = [np.pi / 4,  65 / 180 * np.pi, ]
Rxs = [Rx1, Rx1, -Rx1, -Rx1]
Rys = [Ry1, -Ry1, Ry1, -Ry1]
rxs = [rx1, -rx1, rx1, -rx1]
rys = [-ry1, -ry1, ry1, ry1]

In [9]:
forward_matrix = np.zeros((4, 3), dtype=np.float32)
for i in range(4):
    forward_matrix[i] = np.array([rxs[i], rys[i], Rxs[i] * rys[i] - Rys[i] * rxs[i]])
forward_matrix = forward_matrix / rwheel
forward_matrix[:, 0:2] = forward_matrix[:, 0:2] * 1000 

In [10]:
forward_matrix * np.array([1, -1, 1, -1])[:, np.newaxis]

array([[ 23.57022667, -23.57022667,  -5.5485487 ],
       [ 23.57022667,  23.57022667,   5.5485487 ],
       [ 23.57022667,  23.57022667,  -5.5485487 ],
       [ 23.57022667, -23.57022667,   5.5485487 ]])

# Forward kinematic

In [7]:
np.linalg.lstsq(forward_matrix, np.eye(4))

NameError: name 'forward_matrix' is not defined